# IMPORTS

In [6]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
from parse import compile

# Part 1

In [28]:
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    _, times = data[0].split((":"))
    times = list(map(int, times.split()))
    _, distance = data[1].split(":")
    distance = list(map(int, distance.split()))
    res = 1
    for t, d in zip(times, distance):
        ways = 0
        for spd in range(t + 1):
            if spd * (t - spd) > d: ways += 1
        res *= ways
    print(res)


4403592


# Part 2

In [36]:
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    _, times = data[0].split((":"))
    time = int("".join(times.split()))
    _, distances = data[1].split(":")
    distance = int("".join(distances.split()))
    res = sum(1 for t in range(time + 1) if t * (time - t) > distance)
    print(res)


38017587


In [40]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))

17836115
